# 🏥 Clinical Data and EHR: Hands-on Practice

## Table of Contents
1. [Working with ICD-10 Codes](#practice-1-working-with-icd-10-codes)
2. [HL7 FHIR Data Parsing](#practice-2-hl7-fhir-data-parsing)
3. [Clinical Phenotyping Algorithm](#practice-3-clinical-phenotyping-algorithm)
4. [Clinical NLP: Named Entity Recognition](#practice-4-clinical-nlp-named-entity-recognition)
5. [De-identification of Clinical Notes](#practice-5-de-identification-of-clinical-notes)
6. [OMOP CDM Query Practice](#practice-6-omop-cdm-query-practice)

## Installing and Importing Essential Libraries

In [ ]:
# Install required packages (run only once)
# !pip install pandas numpy matplotlib seaborn scikit-learn

# Import essential libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
import re
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Visualization settings
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 11
sns.set_style('whitegrid')

print("✅ All libraries loaded successfully!")

---
## Practice 1: Working with ICD-10 Codes

### 🎯 Learning Objectives
- Understand ICD-10 code structure
- Parse and validate diagnosis codes
- Group codes by category

### 📖 Key Concepts
**ICD-10 Structure:** 3-7 alphanumeric characters
- Chapter (A-Z)
- Body system/condition
- Laterality & severity

In [ ]:
# 1.1 Create sample patient diagnosis data
def create_diagnosis_data():
    """Create sample diagnosis data with ICD-10 codes"""
    
    diagnoses = {
        'patient_id': [1001, 1002, 1003, 1004, 1005, 1006, 1007, 1008],
        'icd10_code': ['E11.9', 'I10', 'J45.909', 'E11.65', 'I50.9', 'E78.5', 'J44.1', 'I25.10'],
        'description': [
            'Type 2 diabetes mellitus without complications',
            'Essential hypertension',
            'Unspecified asthma, uncomplicated',
            'Type 2 diabetes with hyperglycemia',
            'Heart failure, unspecified',
            'Hyperlipidemia, unspecified',
            'COPD with acute exacerbation',
            'Atherosclerotic heart disease'
        ],
        'diagnosis_date': pd.date_range('2024-01-01', periods=8, freq='W')
    }
    
    df = pd.DataFrame(diagnoses)
    return df

# Create and display data
df_diagnoses = create_diagnosis_data()
print("Sample Patient Diagnosis Data:")
print(df_diagnoses)
print(f"\nTotal records: {len(df_diagnoses)}")

In [ ]:
# 1.2 ICD-10 code parsing and categorization
def parse_icd10_code(code):
    """Parse ICD-10 code and extract chapter"""
    
    # ICD-10 Chapter mapping (simplified)
    chapter_map = {
        'A': 'Infectious diseases', 'B': 'Infectious diseases',
        'C': 'Neoplasms', 'D': 'Blood diseases',
        'E': 'Endocrine/nutritional/metabolic',
        'F': 'Mental disorders',
        'G': 'Nervous system', 'H': 'Eye/ear',
        'I': 'Circulatory system',
        'J': 'Respiratory system',
        'K': 'Digestive system',
        'L': 'Skin', 'M': 'Musculoskeletal',
        'N': 'Genitourinary', 'O': 'Pregnancy',
        'P': 'Perinatal', 'Q': 'Congenital',
        'R': 'Symptoms/signs', 'S': 'Injury',
        'T': 'Injury', 'V': 'External causes',
        'W': 'External causes', 'X': 'External causes',
        'Y': 'External causes', 'Z': 'Health status'
    }
    
    chapter = code[0]
    category = code[:3]
    
    return {
        'chapter': chapter,
        'chapter_name': chapter_map.get(chapter, 'Unknown'),
        'category': category,
        'full_code': code
    }

# Apply parsing
df_diagnoses['chapter'] = df_diagnoses['icd10_code'].apply(lambda x: parse_icd10_code(x)['chapter'])
df_diagnoses['chapter_name'] = df_diagnoses['icd10_code'].apply(lambda x: parse_icd10_code(x)['chapter_name'])
df_diagnoses['category'] = df_diagnoses['icd10_code'].apply(lambda x: parse_icd10_code(x)['category'])

print("Parsed ICD-10 Data:")
print(df_diagnoses[['patient_id', 'icd10_code', 'chapter', 'chapter_name']])

# Visualize distribution
plt.figure(figsize=(10, 5))
df_diagnoses['chapter_name'].value_counts().plot(kind='barh', color='#1E64C8')
plt.title('Distribution of Diagnoses by Body System', fontsize=14, fontweight='bold')
plt.xlabel('Count')
plt.tight_layout()
plt.show()

---
## Practice 2: HL7 FHIR Data Parsing

### 🎯 Learning Objectives
- Understand FHIR resource structure
- Parse JSON-formatted FHIR data
- Extract key patient information

### 📖 Key Concepts
**FHIR Resources:** Patient, Observation, Medication, Condition

In [ ]:
# 2.1 Create sample FHIR Patient resource
def create_fhir_patient():
    """Create a sample FHIR Patient resource"""
    
    fhir_patient = {
        "resourceType": "Patient",
        "id": "example-001",
        "name": [{
            "use": "official",
            "family": "Smith",
            "given": ["John", "Robert"]
        }],
        "gender": "male",
        "birthDate": "1980-05-15",
        "address": [{
            "use": "home",
            "city": "Boston",
            "state": "MA",
            "postalCode": "02115"
        }],
        "identifier": [{
            "system": "http://hospital.org/mrn",
            "value": "MRN-123456"
        }]
    }
    
    return fhir_patient

# Create and display
patient = create_fhir_patient()
print("FHIR Patient Resource:")
print(json.dumps(patient, indent=2))

In [ ]:
# 2.2 Parse FHIR data and extract information
def parse_fhir_patient(fhir_resource):
    """Extract key information from FHIR Patient resource"""
    
    # Extract name
    name = fhir_resource.get('name', [{}])[0]
    full_name = f"{' '.join(name.get('given', []))} {name.get('family', '')}"
    
    # Extract identifier (MRN)
    identifiers = fhir_resource.get('identifier', [])
    mrn = identifiers[0].get('value', 'N/A') if identifiers else 'N/A'
    
    # Calculate age
    birth_date = datetime.strptime(fhir_resource.get('birthDate', '2000-01-01'), '%Y-%m-%d')
    age = (datetime.now() - birth_date).days // 365
    
    # Extract address
    address = fhir_resource.get('address', [{}])[0]
    city = address.get('city', 'N/A')
    state = address.get('state', 'N/A')
    
    patient_info = {
        'MRN': mrn,
        'Name': full_name,
        'Gender': fhir_resource.get('gender', 'N/A'),
        'Age': age,
        'Birth Date': fhir_resource.get('birthDate', 'N/A'),
        'City': city,
        'State': state
    }
    
    return patient_info

# Parse and display
patient_info = parse_fhir_patient(patient)
print("\nExtracted Patient Information:")
print("=" * 50)
for key, value in patient_info.items():
    print(f"{key:15s}: {value}")

---
## Practice 3: Clinical Phenotyping Algorithm

### 🎯 Learning Objectives
- Implement a simple phenotyping algorithm
- Apply inclusion/exclusion criteria
- Identify patient cohorts

### 📖 Key Concepts
**Phenotype:** ICD codes + Labs + Medications + Temporal logic

In [ ]:
# 3.1 Create comprehensive patient data
def create_patient_cohort():
    """Create sample patient data for phenotyping"""
    
    np.random.seed(42)
    n_patients = 50
    
    patients = {
        'patient_id': range(1001, 1001 + n_patients),
        'age': np.random.randint(30, 80, n_patients),
        'gender': np.random.choice(['M', 'F'], n_patients),
        'has_diabetes_dx': np.random.choice([True, False], n_patients, p=[0.3, 0.7]),
        'hba1c': np.random.uniform(5.0, 10.0, n_patients),
        'has_metformin': np.random.choice([True, False], n_patients, p=[0.25, 0.75]),
        'num_encounters': np.random.randint(1, 10, n_patients)
    }
    
    df = pd.DataFrame(patients)
    return df

# Create cohort
df_cohort = create_patient_cohort()
print("Patient Cohort Data (first 10 patients):")
print(df_cohort.head(10))
print(f"\nTotal patients: {len(df_cohort)}")

In [ ]:
# 3.2 Implement Type 2 Diabetes phenotyping algorithm
def identify_diabetes_cohort(df):
    """Identify Type 2 Diabetes patients using phenotype algorithm"""
    
    print("Type 2 Diabetes Phenotyping Algorithm")
    print("=" * 60)
    
    # Inclusion criteria
    print("\nInclusion Criteria:")
    print("1. Has diabetes diagnosis (ICD-10: E11.x)")
    print("2. HbA1c >= 6.5%")
    print("3. Age >= 18")
    
    # Apply criteria
    criteria_1 = df['has_diabetes_dx'] == True
    criteria_2 = df['hba1c'] >= 6.5
    criteria_3 = df['age'] >= 18
    
    # Combine all criteria
    diabetes_cohort = df[criteria_1 & criteria_2 & criteria_3].copy()
    
    print(f"\nResults:")
    print(f"  Patients with diabetes diagnosis: {criteria_1.sum()}")
    print(f"  Patients with HbA1c >= 6.5%: {criteria_2.sum()}")
    print(f"  Patients age >= 18: {criteria_3.sum()}")
    print(f"\n✅ Final cohort size: {len(diabetes_cohort)} patients")
    
    # Additional stratification
    diabetes_cohort['on_metformin'] = diabetes_cohort['has_metformin']
    diabetes_cohort['controlled'] = diabetes_cohort['hba1c'] < 7.0
    
    print(f"\nCohort Characteristics:")
    print(f"  On Metformin: {diabetes_cohort['on_metformin'].sum()} ({diabetes_cohort['on_metformin'].sum()/len(diabetes_cohort)*100:.1f}%)")
    print(f"  Well-controlled (HbA1c < 7%): {diabetes_cohort['controlled'].sum()} ({diabetes_cohort['controlled'].sum()/len(diabetes_cohort)*100:.1f}%)")
    
    return diabetes_cohort

# Identify cohort
diabetes_patients = identify_diabetes_cohort(df_cohort)

# Visualize
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# HbA1c distribution
axes[0].hist(diabetes_patients['hba1c'], bins=15, color='#1E64C8', edgecolor='white')
axes[0].axvline(7.0, color='red', linestyle='--', linewidth=2, label='Control threshold (7.0%)')
axes[0].set_xlabel('HbA1c (%)')
axes[0].set_ylabel('Number of Patients')
axes[0].set_title('HbA1c Distribution in Diabetes Cohort')
axes[0].legend()

# Treatment status
treatment_counts = diabetes_patients['on_metformin'].value_counts()
axes[1].pie(treatment_counts, labels=['Not on Metformin', 'On Metformin'], 
            autopct='%1.1f%%', colors=['#6ba3e0', '#1E64C8'], startangle=90)
axes[1].set_title('Metformin Treatment Status')

plt.tight_layout()
plt.show()

---
## Practice 4: Clinical NLP - Named Entity Recognition

### 🎯 Learning Objectives
- Extract medical entities from clinical text
- Identify diseases, medications, and symptoms
- Apply rule-based NER

### 📖 Key Concepts
**NER:** Identifying and classifying named entities (diseases, drugs, symptoms)

In [ ]:
# 4.1 Create sample clinical notes
clinical_notes = [
    "Patient presents with chest pain and shortness of breath. History of hypertension and diabetes mellitus. Currently on metformin 500mg twice daily and lisinopril 10mg daily.",
    "67-year-old male with COPD exacerbation. Prescribed albuterol inhaler and prednisone 40mg for 5 days. Follow-up in 1 week.",
    "Patient reports severe headache and nausea for 3 days. No history of migraine. Vital signs: BP 145/92, HR 78. Started on sumatriptan 50mg as needed.",
    "Annual physical exam. Patient has well-controlled type 2 diabetes with HbA1c of 6.8%. Continue current regimen of metformin and lifestyle modifications."
]

print("Sample Clinical Notes:")
print("=" * 80)
for i, note in enumerate(clinical_notes, 1):
    print(f"\nNote {i}:")
    print(note)

In [ ]:
# 4.2 Simple rule-based NER
def simple_medical_ner(text):
    """Extract medical entities using simple pattern matching"""
    
    # Define medical term dictionaries
    diseases = ['hypertension', 'diabetes', 'diabetes mellitus', 'copd', 'migraine', 
                'asthma', 'heart failure', 'pneumonia']
    
    medications = ['metformin', 'lisinopril', 'albuterol', 'prednisone', 'sumatriptan',
                   'aspirin', 'insulin', 'atorvastatin']
    
    symptoms = ['chest pain', 'shortness of breath', 'headache', 'nausea', 
                'fever', 'cough', 'fatigue', 'dizziness']
    
    # Convert to lowercase for matching
    text_lower = text.lower()
    
    # Find entities
    found_entities = {
        'diseases': [],
        'medications': [],
        'symptoms': []
    }
    
    for disease in diseases:
        if disease in text_lower:
            found_entities['diseases'].append(disease)
    
    for medication in medications:
        if medication in text_lower:
            # Extract dosage if present
            dosage_pattern = rf'{medication}\s+(\d+\s*mg)'
            dosage_match = re.search(dosage_pattern, text_lower)
            if dosage_match:
                found_entities['medications'].append(f"{medication} {dosage_match.group(1)}")
            else:
                found_entities['medications'].append(medication)
    
    for symptom in symptoms:
        if symptom in text_lower:
            found_entities['symptoms'].append(symptom)
    
    return found_entities

# Apply NER to all notes
print("\nNamed Entity Recognition Results:")
print("=" * 80)

all_entities = []
for i, note in enumerate(clinical_notes, 1):
    entities = simple_medical_ner(note)
    all_entities.append(entities)
    
    print(f"\nNote {i}:")
    print(f"  Diseases found: {', '.join(entities['diseases']) if entities['diseases'] else 'None'}")
    print(f"  Medications found: {', '.join(entities['medications']) if entities['medications'] else 'None'}")
    print(f"  Symptoms found: {', '.join(entities['symptoms']) if entities['symptoms'] else 'None'}")

# Aggregate statistics
print("\n" + "=" * 80)
print("Summary Statistics:")
total_diseases = sum(len(e['diseases']) for e in all_entities)
total_medications = sum(len(e['medications']) for e in all_entities)
total_symptoms = sum(len(e['symptoms']) for e in all_entities)

print(f"  Total diseases extracted: {total_diseases}")
print(f"  Total medications extracted: {total_medications}")
print(f"  Total symptoms extracted: {total_symptoms}")

---
## Practice 5: De-identification of Clinical Notes

### 🎯 Learning Objectives
- Identify PHI (Protected Health Information)
- Remove or mask identifiable information
- Apply Safe Harbor method

### 📖 Key Concepts
**PHI:** Names, dates, addresses, MRNs, phone numbers (18 HIPAA identifiers)

In [ ]:
# 5.1 Sample clinical note with PHI
clinical_note_with_phi = """
Patient Name: John Robert Smith
DOB: 05/15/1980
MRN: 123456789
Address: 123 Main Street, Boston, MA 02115
Phone: (617) 555-1234
Email: john.smith@email.com

Chief Complaint: Chest pain

HPI: 44-year-old male presents on 11/14/2024 with acute onset chest pain. 
Patient reports pain started around 2:30 PM while at work. 
Past medical history significant for hypertension diagnosed in 2015.

Current Medications:
- Metformin 500mg BID
- Lisinopril 10mg daily

Assessment: Rule out acute coronary syndrome
Plan: Admit to cardiology service, Dr. Jane Williams
"""

print("Original Clinical Note with PHI:")
print("=" * 80)
print(clinical_note_with_phi)

In [ ]:
# 5.2 Simple de-identification function
def deidentify_clinical_note(text):
    """Remove PHI from clinical notes using pattern matching"""
    
    deidentified = text
    
    # 1. Remove names
    deidentified = re.sub(r'Patient Name:.*', 'Patient Name: [REDACTED]', deidentified)
    deidentified = re.sub(r'Dr\.\s+[A-Z][a-z]+\s+[A-Z][a-z]+', 'Dr. [REDACTED]', deidentified)
    
    # 2. Remove dates (MM/DD/YYYY format)
    deidentified = re.sub(r'\b\d{1,2}/\d{1,2}/\d{4}\b', '[DATE]', deidentified)
    
    # 3. Remove DOB
    deidentified = re.sub(r'DOB:.*', 'DOB: [REDACTED]', deidentified)
    
    # 4. Remove MRN
    deidentified = re.sub(r'MRN:.*', 'MRN: [REDACTED]', deidentified)
    
    # 5. Remove addresses
    deidentified = re.sub(r'Address:.*', 'Address: [REDACTED]', deidentified)
    
    # 6. Remove phone numbers
    deidentified = re.sub(r'Phone:.*', 'Phone: [REDACTED]', deidentified)
    deidentified = re.sub(r'\(\d{3}\)\s*\d{3}-\d{4}', '[PHONE]', deidentified)
    
    # 7. Remove email addresses
    deidentified = re.sub(r'Email:.*', 'Email: [REDACTED]', deidentified)
    deidentified = re.sub(r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b', '[EMAIL]', deidentified)
    
    # 8. Generalize ages >89
    def generalize_age(match):
        age = int(match.group(1))
        if age > 89:
            return ">89-year-old"
        return match.group(0)
    
    deidentified = re.sub(r'(\d+)-year-old', generalize_age, deidentified)
    
    # 9. Remove specific times
    deidentified = re.sub(r'\b\d{1,2}:\d{2}\s*[AP]M\b', '[TIME]', deidentified)
    
    return deidentified

# Apply de-identification
deidentified_note = deidentify_clinical_note(clinical_note_with_phi)

print("\nDe-identified Clinical Note:")
print("=" * 80)
print(deidentified_note)

# Count redactions
redaction_count = deidentified_note.count('[REDACTED]') + deidentified_note.count('[DATE]') + \
                  deidentified_note.count('[PHONE]') + deidentified_note.count('[EMAIL]') + \
                  deidentified_note.count('[TIME]')

print("\n" + "=" * 80)
print(f"✅ De-identification complete: {redaction_count} PHI elements removed/masked")

---
## Practice 6: OMOP CDM Query Practice

### 🎯 Learning Objectives
- Understand OMOP Common Data Model structure
- Write queries for cohort identification
- Analyze standardized clinical data

### 📖 Key Concepts
**OMOP Tables:** Person, Condition_Occurrence, Drug_Exposure, Measurement

In [ ]:
# 6.1 Create sample OMOP-like data structure
def create_omop_sample_data():
    """Create sample data mimicking OMOP CDM structure"""
    
    np.random.seed(42)
    
    # Person table
    n_persons = 20
    person = pd.DataFrame({
        'person_id': range(1, n_persons + 1),
        'gender_concept_id': np.random.choice([8507, 8532], n_persons),
        'year_of_birth': np.random.randint(1950, 2000, n_persons),
        'race_concept_id': np.random.choice([8527, 8516, 8515], n_persons)
    })
    
    # Condition_Occurrence table
    diabetes_patients = np.random.choice(person['person_id'], size=8, replace=False)
    condition_occurrence = pd.DataFrame({
        'condition_occurrence_id': range(1, len(diabetes_patients) + 1),
        'person_id': diabetes_patients,
        'condition_concept_id': [201826] * len(diabetes_patients),
        'condition_start_date': pd.date_range('2023-01-01', periods=len(diabetes_patients), freq='M')
    })
    
    # Drug_Exposure table
    metformin_patients = np.random.choice(diabetes_patients, size=5, replace=False)
    drug_exposure = pd.DataFrame({
        'drug_exposure_id': range(1, len(metformin_patients) + 1),
        'person_id': metformin_patients,
        'drug_concept_id': [1503297] * len(metformin_patients),
        'drug_exposure_start_date': pd.date_range('2023-02-01', periods=len(metformin_patients), freq='M')
    })
    
    # Measurement table
    measurement = pd.DataFrame({
        'measurement_id': range(1, len(diabetes_patients) + 1),
        'person_id': diabetes_patients,
        'measurement_concept_id': [3004410] * len(diabetes_patients),
        'measurement_date': pd.date_range('2023-06-01', periods=len(diabetes_patients), freq='M'),
        'value_as_number': np.random.uniform(6.0, 9.5, len(diabetes_patients))
    })
    
    return person, condition_occurrence, drug_exposure, measurement

# Create tables
person_table, condition_table, drug_table, measurement_table = create_omop_sample_data()

print("OMOP CDM Sample Data")
print("=" * 80)
print("\nPerson Table (first 5 rows):")
print(person_table.head())
print("\nCondition_Occurrence Table:")
print(condition_table)
print("\nDrug_Exposure Table:")
print(drug_table)
print("\nMeasurement Table (HbA1c):")
print(measurement_table.head())

In [ ]:
# 6.2 Query: Identify diabetes patients on metformin with recent HbA1c
def omop_cohort_query():
    """SQL-like query using pandas to identify cohort"""
    
    print("OMOP Cohort Query: Diabetes + Metformin + HbA1c")
    print("=" * 80)
    
    # Step 1: Get diabetes patients
    diabetes_persons = condition_table['person_id'].unique()
    print(f"\nStep 1: Patients with Type 2 Diabetes: {len(diabetes_persons)}")
    
    # Step 2: Filter for those on metformin
    metformin_persons = drug_table['person_id'].unique()
    diabetes_on_metformin = set(diabetes_persons) & set(metformin_persons)
    print(f"Step 2: Diabetes patients on Metformin: {len(diabetes_on_metformin)}")
    
    # Step 3: Get HbA1c measurements
    cohort_measurements = measurement_table[
        measurement_table['person_id'].isin(diabetes_on_metformin)
    ].copy()
    print(f"Step 3: Patients with HbA1c measurements: {len(cohort_measurements)}")
    
    # Step 4: Join with person demographics
    cohort = cohort_measurements.merge(
        person_table[['person_id', 'gender_concept_id', 'year_of_birth']],
        on='person_id'
    )
    
    cohort['age'] = 2024 - cohort['year_of_birth']
    cohort['gender'] = cohort['gender_concept_id'].map({8507: 'Male', 8532: 'Female'})
    cohort['controlled'] = cohort['value_as_number'] < 7.0
    
    print(f"\n✅ Final cohort: {len(cohort)} patients")
    print("\nCohort Characteristics:")
    print(f"  Mean age: {cohort['age'].mean():.1f} years")
    print(f"  Mean HbA1c: {cohort['value_as_number'].mean():.2f}%")
    print(f"  Well-controlled (<7%): {cohort['controlled'].sum()} ({cohort['controlled'].sum()/len(cohort)*100:.1f}%)")
    print(f"\nGender distribution:")
    print(cohort['gender'].value_counts())
    
    return cohort

# Execute query
final_cohort = omop_cohort_query()

# Visualize results
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# HbA1c by gender
final_cohort.boxplot(column='value_as_number', by='gender', ax=axes[0])
axes[0].set_title('HbA1c Distribution by Gender')
axes[0].set_xlabel('Gender')
axes[0].set_ylabel('HbA1c (%)')
axes[0].axhline(y=7.0, color='red', linestyle='--', label='Control threshold')
plt.sca(axes[0])
plt.xticks(rotation=0)

# Age distribution
axes[1].hist(final_cohort['age'], bins=10, color='#1E64C8', edgecolor='white')
axes[1].set_xlabel('Age (years)')
axes[1].set_ylabel('Number of Patients')
axes[1].set_title('Age Distribution in Cohort')

plt.tight_layout()
plt.show()

print("\nCohort Details:")
print(final_cohort[['person_id', 'age', 'gender', 'value_as_number', 'controlled']].to_string(index=False))

---
## 🎯 Practice Complete!

### Summary of What We Learned:

1. **ICD-10 Coding**: Parsing and categorizing diagnosis codes
2. **FHIR Data**: Working with standardized healthcare data formats
3. **Clinical Phenotyping**: Implementing algorithms to identify patient cohorts
4. **Clinical NLP**: Extracting medical entities from unstructured text
5. **De-identification**: Removing PHI to protect patient privacy
6. **OMOP CDM**: Querying standardized observational health data

### Key Insights:
- Clinical data requires specialized handling and domain knowledge
- Standardization (ICD-10, FHIR, OMOP) enables interoperability
- Privacy and security are paramount in healthcare analytics
- Combining structured and unstructured data provides comprehensive insights

### Next Steps:
- Advanced NLP with BioBERT or Clinical BERT
- Machine learning for risk prediction
- Real-world evidence generation
- Integration with EHR systems

### Resources:
- FHIR Specification: https://www.hl7.org/fhir/
- OHDSI OMOP CDM: https://www.ohdsi.org/data-standardization/
- HIPAA Privacy Rule: https://www.hhs.gov/hipaa/
- Clinical NLP Tools: scispaCy, MedCAT, BioBERT